<a href="https://colab.research.google.com/github/skankekarucdavis/learning-git/blob/master/lightFM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp311-cp311-linux_x86_64.whl size=831161 sha256=230f975e6130292cb4483a2d386cbc650c9daea2f1dc27da8db4b17fb48b49d2
  Stored in directory: /root/.cache/pip/wheels/b9/0d/8a/0729d2e6e3ca2a898ba55201f905da7db3f838a33df5b3fcdd
Successfully built lightfm


In [2]:
import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix
from lightfm import LightFM
from lightfm.evaluation import auc_score
from sklearn.model_selection import train_test_split

In [4]:
df=pd.read_csv("/content/merged2.csv")
df.head()

,Unnamed: 0,Unnamed: 0_inventory,Unnamed: 0.1,Product ID,SKU_inventory,Variant ID,Quantity,Color,Tags,Product Type,...,Email,SKU_order,State,Paid at,Prints_order,Neck_order,Sleeves_order,Extracted Tags_order,url,product_url
0,0,20,1910,1948244751137,AB2620PM4423 SANGRIAXS,1938244694336,1,Sangria,"short-sleeve, tops",Tops,...,206344,AB2620PM4423 SANGRIAXS,FL,NaN,Solid,Round Neck,Short Sleeve,Solid Round Neck Short Sleeve,https://www.fashom.com/,https://cdn.shopify.com/s/files/1/2503/4628/pr...
1,1,21,1975,1948244171800,AB22237413 CAMEL/NAVYXS,1938244496106,1,Camel/Navy,"cashmere, fallcatalog, finalsale, long sleeve,...",Tops,...,123946,AB22237413 CAMEL/NAVYXS,MO,NaN,Solid,Turtle Neck,Full Sleeve,Solid Turtle Neck Full Sleeve,https://www.fashom.com/,https://cdn.shopify.com/s/files/1/2503/4628/pr...
2,2,21,1975,1948244171800,AB22237413 CAMEL/NAVYXS,1938244496106,1,Camel/Navy,"cashmere, fallcatalog, finalsale, long sleeve,...",Tops,...,404022,AB22237413 CAMEL/NAVYXS,IL,NaN,Solid,Turtle Neck,Full Sleeve,Solid Turtle Neck Full Sleeve,https://www.fashom.com/,https://cdn.shopify.com/s/files/1/2503/4628/pr...
3,3,21,1975,1948244171800,AB22237413 CAMEL/NAVYXS,1938244496106,1,Camel/Navy,"cashmere, fallcatalog, finalsale, long sleeve,...",Tops,...,159388,AB22237413 CAMEL/NAVYXS,WY,11/9/2022 7:56,Solid,Turtle Neck,Full Sleeve,Solid Turtle Neck Full Sleeve,https://www.fashom.com/,https://cdn.shopify.com/s/files/1/2503/4628/pr...
4,4,21,1975,1948244171800,AB22237413 CAMEL/NAVYXS,1938244496106,1,Camel/Navy,"cashmere, fallcatalog, finalsale, long sleeve,...",Tops,...,306701,AB22237413 CAMEL/NAVYXS,FL,9/14/2022 14:14,Solid,Turtle Neck,Full Sleeve,Solid Turtle Neck Full Sleeve,https://www.fashom.com/,https://cdn.shopify.com/s/files/1/2503/4628/pr...


In [5]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [6]:

def dense_to_sparse(df, user_col='user_id', item_col='product_id', rating_col='purchase',
                 user_mapping=None, item_mapping=None):

    if user_mapping is None:
        user_mapping = {id: i for i, id in enumerate(df[user_col].unique())}
    if item_mapping is None:
        item_mapping = {id: i for i, id in enumerate(df[item_col].unique())}


    # Map user and item IDs to matrix indices
    row = df[user_col].map(user_mapping)
    col = df[item_col].map(item_mapping)
    data = df[rating_col].values
    #Shape of the matrix
    shape = (len(user_mapping), len(item_mapping))
    matrix = coo_matrix((data, (row, col)), shape=shape)
    return  matrix, user_mapping, item_mapping





In [7]:


interaction_matrix, user_mapping, item_mapping = dense_to_sparse(df,"requester_id","Variant ID","Quantity")
interaction_matrix



<1123x99 sparse matrix of type '<class 'numpy.int64'>'
	with 1214 stored elements in COOrdinate format>

In [8]:
train_matrix,_,_=dense_to_sparse(train_df,"requester_id","Variant ID","Quantity",user_mapping=user_mapping,item_mapping=item_mapping)
train_matrix.shape

(1123, 99)

In [9]:
test_matrix,_,_=dense_to_sparse(test_df,"requester_id","Variant ID","Quantity",user_mapping=user_mapping,item_mapping=item_mapping)
test_matrix.shape

(1123, 99)

In [10]:
model = LightFM(no_components=30, loss='logistic')
model.fit(train_matrix, epochs=30, verbose=True)

# Evaluate the model
train_auc = auc_score(model, train_matrix).mean()

train_auc

Epoch: 100%|██████████| 30/30 [00:00<00:00, 915.34it/s]


0.79190606

In [11]:
test_auc = auc_score(model, test_matrix).mean()


In [12]:
test_auc
#check on the number on epochs and no of components in a for loop

0.75426483

In [14]:
from lightfm import LightFM
from lightfm.evaluation import auc_score
from lightfm.cross_validation import random_train_test_split
import numpy as np
from sklearn.model_selection import ParameterGrid

# Assume train_matrix is your user-item interaction matrix (sparse)

# Define hyperparameter grid
param_grid = {
    'no_components': [10, 20, 30, 50],
    'epochs': [10, 20, 30],
    'loss': ['logistic']  # Add more loss functions if desired
}

# Number of folds
k_folds = 5
results = []

# Loop over hyperparameter combinations
for params in ParameterGrid(param_grid):
    fold_train_aucs = []
    fold_test_aucs = []

    print(f"\nTesting Params: {params}")

    # Simulate k-fold by repeatedly splitting the data
    for fold in range(k_folds):
        # Split data into train and test (validation) sets
        train, test = random_train_test_split(
            train_matrix,
            test_percentage=0.2,
            random_state=fold
        )

        # Initialize model with current hyperparameters
        model = LightFM(
            no_components=params['no_components'],
            loss=params['loss']
        )

        # Fit the model
        model.fit(train, epochs=params['epochs'], verbose=False)

        # Evaluate on both train and test sets
        train_auc = auc_score(model, train).mean()
        test_auc = auc_score(model, test).mean()

        fold_train_aucs.append(train_auc)
        fold_test_aucs.append(test_auc)

        # Print AUC for this fold
        print(f"Fold {fold + 1}: Train AUC = {train_auc:.4f}, Test AUC = {test_auc:.4f}")

    # Calculate average AUCs across folds
    avg_train_auc = np.mean(fold_train_aucs)
    avg_test_auc = np.mean(fold_test_aucs)

    results.append({
        'params': params,
        'avg_train_auc': avg_train_auc,
        'avg_test_auc': avg_test_auc
    })

    # Print average results for this combination
    print(f"Avg Train AUC: {avg_train_auc:.4f}, Avg Test AUC: {avg_test_auc:.4f}")

# Find the best hyperparameters based on test AUC
best_result = max(results, key=lambda x: x['avg_test_auc'])
print("\nBest Hyperparameters (based on Avg Test AUC):", best_result['params'])
print("Best Avg Train AUC:", best_result['avg_train_auc'])
print("Best Avg Test AUC:", best_result['avg_test_auc'])

# Train final model with best hyperparameters
best_model = LightFM(
    no_components=best_result['params']['no_components'],
    loss=best_result['params']['loss']
)
best_model.fit(train_matrix, epochs=best_result['params']['epochs'], verbose=True)

# Final evaluation on full train_matrix
final_train_auc = auc_score(best_model, train_matrix).mean()
print("Final Train AUC on full data:", final_train_auc)


Testing Params: {'epochs': 10, 'loss': 'logistic', 'no_components': 10}
Fold 1: Train AUC = 0.8001, Test AUC = 0.7175
Fold 2: Train AUC = 0.7936, Test AUC = 0.7425
Fold 3: Train AUC = 0.7924, Test AUC = 0.7473
Fold 4: Train AUC = 0.8019, Test AUC = 0.7071
Fold 5: Train AUC = 0.7982, Test AUC = 0.7308
Avg Train AUC: 0.7972, Avg Test AUC: 0.7291

Testing Params: {'epochs': 10, 'loss': 'logistic', 'no_components': 20}
Fold 1: Train AUC = 0.8011, Test AUC = 0.7178
Fold 2: Train AUC = 0.7944, Test AUC = 0.7454
Fold 3: Train AUC = 0.7926, Test AUC = 0.7465
Fold 4: Train AUC = 0.8035, Test AUC = 0.7095
Fold 5: Train AUC = 0.7991, Test AUC = 0.7300
Avg Train AUC: 0.7981, Avg Test AUC: 0.7299

Testing Params: {'epochs': 10, 'loss': 'logistic', 'no_components': 30}
Fold 1: Train AUC = 0.8012, Test AUC = 0.7192
Fold 2: Train AUC = 0.7943, Test AUC = 0.7457
Fold 3: Train AUC = 0.7920, Test AUC = 0.7473
Fold 4: Train AUC = 0.8028, Test AUC = 0.7078
Fold 5: Train AUC = 0.7981, Test AUC = 0.7300
Avg

Epoch: 100%|██████████| 20/20 [00:00<00:00, 508.17it/s]

Final Train AUC on full data: 0.7905913
